In [1]:
import numpy as np
import pandas as pd
from datasets import load_dataset
df = pd.read_csv("hf://datasets/openadmet/openadmet-expansionrx-challenge-train-data/expansion_data_train_raw.csv")

In [2]:
test = load_dataset("openadmet/openadmet-expansionrx-challenge-test-data-blinded")

In [3]:
test = pd.DataFrame(test['test'])

In [4]:
df_LogD = df[['SMILES', 'LogD']].dropna()
df_KSOL = df[['SMILES', 'KSOL']].dropna()
df_KSOL['lgKSOL'] = np.log10(df_KSOL['KSOL'])
df_HLM = df[['SMILES', 'HLM CLint']].dropna()
df_HLM = df_HLM.replace(0, 0.01)
df_HLM['lgHLM'] = np.log10(df_HLM['HLM CLint'])
df_MLM = df[['SMILES', 'MLM CLint']].dropna()
df_MLM = df_MLM.replace(0, 0.01)
df_MLM['lgMLM'] = np.log10(df_MLM['MLM CLint'])
df_Caco_2A = df[['SMILES', 'Caco-2 Permeability Papp A>B']].dropna()
df_Caco_2A = df_Caco_2A.replace(0, 0.01)
df_Caco_2A['lgCaco_2A'] = np.log10(df_Caco_2A['Caco-2 Permeability Papp A>B'])
df_Caco_2B = df[['SMILES', 'Caco-2 Permeability Efflux']].dropna()
df_Caco_2B = df_Caco_2B.replace(0, 0.01)
df_Caco_2B['lgCaco_2B'] = np.log10(df_Caco_2B['Caco-2 Permeability Efflux'])
df_MPPB = df[['SMILES', 'MPPB']].dropna()
df_MPPB = df_MPPB.replace(0, 0.01)
df_MPPB['lgMPPB'] = np.log10(df_MPPB['MPPB'])
df_MBPB = df[['SMILES', 'MBPB']].dropna()
df_MBPB = df_MBPB.replace(0, 0.01)
df_MBPB['lgMBPB'] = np.log10(df_MBPB['MBPB'])
df_MGMB = df[['SMILES', 'MGMB']].dropna()
df_MGMB = df_MGMB.replace(0, 0.01)
df_MGMB['lgMGMB'] = np.log10(df_MGMB['MGMB'])

In [5]:
train_sets = {
    'LogD': df_LogD,
    'lgKSOL': df_KSOL,
    'lgHLM': df_HLM,
    'lgMLM': df_MLM,
    'lgCaco_2A': df_Caco_2A,
    'lgCaco_2B': df_Caco_2B,
    'lgMPPB': df_MPPB,
    'lgMBPB': df_MBPB,
    'lgMGMB': df_MGMB,
}

test_set_common = test   

In [6]:
import torch
from chemprop import featurizers, nn
from chemprop.data import BatchMolGraph, MoleculeDatapoint

featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()
agg = nn.MeanAggregation()
chemeleon_mp = torch.load("/home/lab101/temp/mdm/bot/chemeleon_mp.pt", weights_only=True)
mp = nn.BondMessagePassing(**chemeleon_mp['hyper_parameters'])
mp.load_state_dict(chemeleon_mp['state_dict'])

<All keys matched successfully>

In [7]:
from pathlib import Path
from lightning import pytorch as pl
from lightning.pytorch.callbacks import ModelCheckpoint
from chemprop import data, models
from chemprop.data import BatchMolGraph, MoleculeDatapoint
chemprop_dir = Path.cwd().parent

In [8]:
X_dict = {}
X_test_dict = {}
y_dict = {}

featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()
agg = nn.MeanAggregation()

def extract_features_in_batches(
    smis,
    featurizer,
    mp,
    agg,
    batch_size=32
):

    mol_feats_list = []

    mp.eval()
    agg.eval()

    with torch.no_grad():
        for i in range(0, len(smis), batch_size):
            batch_smis = smis[i:i + batch_size]

            batch_datapoints = [
                MoleculeDatapoint.from_smi(smi) for smi in batch_smis
            ]
            batch_mols = [dp.mol for dp in batch_datapoints]
            batch_graphs = [featurizer(mol) for mol in batch_mols]

            batch_mol_graph = BatchMolGraph(batch_graphs)

            atom_embeddings = mp(batch_mol_graph)
            mol_embeddings = agg(atom_embeddings, batch_mol_graph.batch)

            if mol_embeddings.ndim == 3:
                mol_embeddings = mol_embeddings.squeeze(0)

            mol_feats_list.append(mol_embeddings.cpu().numpy())

    return np.concatenate(mol_feats_list, axis=0)

for target, train_df in train_sets.items():
    print(f"\n================ Target: {target} ================")

    
    smis_train = train_df['SMILES'].values.astype(str)
    y = train_df[target].values.astype(np.float32)

  
    smis_test = test_set_common['SMILES'].values.astype(str)

    
    all_data = [
        data.MoleculeDatapoint.from_smi(smi, [yy])
        for smi, yy in zip(smis_train, y)
    ]
    mols = [d.mol for d in all_data]

    train_idx, val_idx, _ = data.make_split_indices(
        mols, "random", (0.9, 0.1, 0.0)
    )

    train_data, val_data, _ = data.split_data_by_indices(
        all_data, train_idx, val_idx, []
    )

    train_dset = data.MoleculeDataset(train_data[0], featurizer)
    scaler = train_dset.normalize_targets()

    val_dset = data.MoleculeDataset(val_data[0], featurizer)
    val_dset.normalize_targets(scaler)

    train_loader = data.build_dataloader(train_dset, num_workers=0)
    val_loader = data.build_dataloader(val_dset, num_workers=0, shuffle=False)

    
    chemeleon_mp = torch.load(
        "/home/lab101/temp/mdm/bot/chemeleon_mp.pt",
        weights_only=True
    )

    mp = nn.BondMessagePassing(**chemeleon_mp['hyper_parameters'])
    mp.load_state_dict(chemeleon_mp['state_dict'])

    ffn = nn.RegressionFFN(
        input_dim=mp.output_dim,
        output_transform=nn.UnscaleTransform.from_standard_scaler(scaler),
    )

    mpnn = models.MPNN(
        mp, agg, ffn,
        batch_norm=False,
        metrics=[nn.metrics.RMSE(), nn.metrics.MAE()]
    )

    ckpt_dir = f"/home/lab101/temp/mdm/bot/Chemeleon_{target}"

    checkpointing = ModelCheckpoint(
        dirpath=ckpt_dir,
        filename="best-{epoch}-{val_loss:.3f}",
        monitor="val_loss",
        mode="min"
    )

    trainer = pl.Trainer(
        logger=False,
        enable_checkpointing=True,
        accelerator="auto",
        devices=1,
        max_epochs=25,
        callbacks=[checkpointing]
    )

    trainer.fit(mpnn, train_loader, val_loader)

    
    best_mpnn = models.MPNN.load_from_checkpoint(
        checkpointing.best_model_path,
        mp=mp,
        agg=agg,
        ffn=ffn,
        batch_norm=False,
        metrics=[nn.metrics.RMSE(), nn.metrics.MAE()]
    ).cpu()

    best_mpnn.message_passing = best_mpnn.message_passing.cpu()
    best_mpnn.agg = best_mpnn.agg.cpu()

    
    feats_train = extract_features_in_batches(
        smis_train,
        featurizer,
        best_mpnn.message_passing,
        best_mpnn.agg
    )

    feature_cols = [f'feature_{i}' for i in range(feats_train.shape[1])]
    X_dict[target] = pd.DataFrame(feats_train, columns=feature_cols)
    y_dict[target] = y

   
    feats_test = extract_features_in_batches(
        smis_test,
        featurizer,
        best_mpnn.message_passing,
        best_mpnn.agg
    )

    X_test_dict[target] = pd.DataFrame(feats_test, columns=feature_cols)


================ Target: LogD ================


The return type of make_split_indices has changed in v2.1 - see help(make_split_indices)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3070') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/home/lab101/anaconda3/envs/chemeleon/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:701: Checkpoint directory /home/lab101/temp/mdm/bot/Chemeleon_LogD exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
Loading `train_dataloader` to estimate number of stepping batches.
/home/lab101/anaconda3/envs/chemeleon/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_

Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

/home/lab101/anaconda3/envs/chemeleon/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=103` in the `DataLoader` to improve performance.


Training: |                                               | 0/? [00:00<?, ?it/s]

...

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=25` reached.


In [9]:
target_columns = ['LogD', 'lgKSOL', 'lgHLM', 'lgMLM', 'lgCaco_2A', 'lgCaco_2B', 'lgMPPB', 'lgMBPB', 'lgMGMB'] 

In [10]:
import torch.nn as nn
from sklearn.preprocessing import QuantileTransformer
import rtdl_num_embeddings
import tabm
import math, random
from copy import deepcopy
from tqdm import trange

In [11]:
seed = 1
random.seed(seed)
np.random.seed(seed+1)
torch.manual_seed(seed+2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

k_submodels = 32               
batch_size = 32               
max_epochs = 1000              
patience = 16                  
lr = 1e-3                      
weight_decay = 1e-4
gradient_clipping = 1.0

n_layers = 3
hidden = 256
dropout = 0.1

In [12]:
def make_piecewise_embeddings(X_train_tensor, n_bins=16, d_embedding=16):

    bins = rtdl_num_embeddings.compute_bins(torch.as_tensor(X_train_tensor, dtype=torch.float32, device='cpu'), n_bins=n_bins)
    emb = rtdl_num_embeddings.PiecewiseLinearEmbeddings(bins, d_embedding=d_embedding, activation=False, version='B')
    return emb

In [13]:
test_predictions = {}

for target in target_columns:

    print(f"\n==============================")
    print(f"Training ensemble for target: {target}")
    print(f"==============================")

    X_train_np = X_dict[target].values.astype(np.float32)
    y_train_np = np.asarray(y_dict[target], dtype=np.float32).reshape(-1)
    X_test_np  = X_test_dict[target].values.astype(np.float32)

    final_seeds = [1, 7, 42, 1337, 2024]

    final_models = []
    final_qts = []
    final_y_stats = []

    for seed in final_seeds:
        print(f"\n--- Seed {seed} ---")

        random.seed(seed)
        np.random.seed(seed + 1)
        torch.manual_seed(seed + 2)

        
        idx = np.random.permutation(len(X_train_np))
        n_val = int(0.1 * len(idx))
        train_idx, val_idx = idx[:-n_val], idx[-n_val:]

        X_tr, X_val = X_train_np[train_idx], X_train_np[val_idx]
        y_tr, y_val = y_train_np[train_idx], y_train_np[val_idx]

        n_quantiles = max(min(len(train_idx) // 30, 1000), 10)
        qt = QuantileTransformer(
            n_quantiles=n_quantiles,
            output_distribution='normal',
            subsample=10**9
        )

        noise = np.random.default_rng(seed).normal(0.0, 1e-6, X_tr.shape).astype(X_tr.dtype)
        X_tr_trans = qt.fit_transform(X_tr + noise)
        X_val_trans = qt.transform(X_val)

        y_mean = y_tr.mean()
        y_std = y_tr.std() if y_tr.std() > 0 else 1.0
        y_tr_std = (y_tr - y_mean) / y_std

       
        X_tr_t = torch.tensor(X_tr_trans, dtype=torch.float32, device=device)
        X_val_t = torch.tensor(X_val_trans, dtype=torch.float32, device=device)
        y_tr_t = torch.tensor(y_tr_std, dtype=torch.float32, device=device)

        
        num_embeddings = make_piecewise_embeddings(X_tr_trans, n_bins=12, d_embedding=16)

        model = tabm.TabM.make(
            n_num_features=X_tr_t.shape[1],
            cat_cardinalities=[],
            d_out=1,
            num_embeddings=num_embeddings,
            arch_type='tabm-mini',
            k=k_submodels,
            n_blocks=n_layers,
            d_block=hidden,
            dropout=dropout,
        ).to(device)

        optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
        loss_fn = nn.MSELoss()

        
        best_rmse = math.inf
        best_state = None
        epochs_wo_improve = 0
        epoch = 0

        while epoch < max_epochs and epochs_wo_improve <= patience:
            model.train()
            perm = torch.randperm(len(X_tr_t), device=device)

            for i in range(0, len(perm), batch_size):
                idxb = perm[i:i + batch_size]
                xb, yb = X_tr_t[idxb], y_tr_t[idxb]

                optimizer.zero_grad()
                preds = model(xb, None).squeeze(-1)
                preds_flat = preds.flatten(0, 1)
                yb_rep = yb.repeat_interleave(k_submodels)
                loss = loss_fn(preds_flat, yb_rep)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), gradient_clipping)
                optimizer.step()

            
            model.eval()
            with torch.no_grad():
                preds = []
                for i in range(0, len(X_val_t), 1024):
                    p = model(X_val_t[i:i+1024], None).cpu().numpy().squeeze(-1)
                    preds.append(p.mean(axis=1))
                preds = np.concatenate(preds)
                preds_orig = preds * y_std + y_mean
                rmse = np.sqrt(mean_squared_error(y_val, preds_orig))

            if rmse < best_rmse - 1e-8:
                best_rmse = rmse
                best_state = deepcopy(model.state_dict())
                epochs_wo_improve = 0
            else:
                epochs_wo_improve += 1

            epoch += 1

        model.load_state_dict(best_state)

        final_models.append(model)
        final_qts.append(qt)
        final_y_stats.append((y_mean, y_std))


    test_preds_all = []

    for model, qt, (y_mean, y_std) in zip(final_models, final_qts, final_y_stats):
        X_test_trans = qt.transform(X_test_np)
        X_test_t = torch.tensor(X_test_trans, dtype=torch.float32, device=device)

        model.eval()
        with torch.no_grad():
            preds = []
            for i in range(0, len(X_test_t), 1024):
                p = model(X_test_t[i:i+1024], None).cpu().numpy().squeeze(-1)
                preds.append(p.mean(axis=1))
            preds = np.concatenate(preds)
            preds_orig = preds * y_std + y_mean
            test_preds_all.append(preds_orig)


    test_predictions[target] = np.mean(test_preds_all, axis=0)

    print(f"Finished target {target}")


Training ensemble for target: LogD

--- Seed 1 ---

--- Seed 7 ---

--- Seed 42 ---

--- Seed 1337 ---

--- Seed 2024 ---
✅ Finished target LogD

Training ensemble for target: lgKSOL

--- Seed 1 ---

--- Seed 7 ---

--- Seed 42 ---

--- Seed 1337 ---

--- Seed 2024 ---
✅ Finished target lgKSOL

Training ensemble for target: lgHLM

--- Seed 1 ---

--- Seed 7 ---

--- Seed 42 ---

--- Seed 1337 ---

--- Seed 2024 ---
✅ Finished target lgHLM

Training ensemble for target: lgMLM

--- Seed 1 ---

--- Seed 7 ---

--- Seed 42 ---

--- Seed 1337 ---

--- Seed 2024 ---
✅ Finished target lgMLM

Training ensemble for target: lgCaco_2A

--- Seed 1 ---

--- Seed 7 ---

--- Seed 42 ---

--- Seed 1337 ---

--- Seed 2024 ---
✅ Finished target lgCaco_2A

Training ensemble for target: lgCaco_2B

--- Seed 1 ---

--- Seed 7 ---

--- Seed 42 ---

--- Seed 1337 ---

--- Seed 2024 ---
✅ Finished target lgCaco_2B

Training ensemble for target: lgMPPB

--- Seed 1 ---

--- Seed 7 ---

--- Seed 42 ---

--- Seed

In [14]:
y_test_KSOL = 10 ** test_predictions['lgKSOL']
y_test_HLM = 10 ** test_predictions['lgHLM']
y_test_MLM = 10 ** test_predictions['lgMLM']
y_test_Caco_2A = 10 ** test_predictions['lgCaco_2A']
y_test_Caco_2B = 10 ** test_predictions['lgCaco_2B']
y_test_MPPB = 10 ** test_predictions['lgMPPB']
y_test_MBPB = 10 ** test_predictions['lgMBPB']
y_test_MGMB = 10 ** test_predictions['lgMGMB']

In [16]:
sub = pd.DataFrame({
    'LogD': test_predictions["LogD"],
    'KSOL': y_test_KSOL,
    'HLM CLint': y_test_HLM,
    'MLM CLint': y_test_MLM,
    'Caco-2 Permeability Papp A>B': y_test_Caco_2A,
    'Caco-2 Permeability Efflux': y_test_Caco_2B,
    'MPPB': y_test_MPPB,
    'MBPB': y_test_MBPB,
    'MGMB': y_test_MGMB
})

In [17]:
submit = pd.concat([test["Molecule Name"], sub], axis=1)

In [19]:
submit.to_csv("tabm.csv", index=False)